### HW_Tarasova_Lesson 2. Keras

1. Обучить, нейронную сеть на Keras на датасете MNIST.

In [1]:
!pip install keras
!pip install mnist
!pip install scikeras[tensorflow]

In [1]:
import numpy as np
import pandas as pd
import mnist
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

### Building model for digit recognition

In [2]:
#import images
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255)
test_images = (test_images / 255)

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

In [3]:
# Build the model.
def create_model(optimizer='adam'):
    model = Sequential([
      Dense(64, input_shape=(784,), activation='relu'),
      Dense(32, activation='relu'),
      Dense(10, activation='softmax')
    ])

    # Compile the model.
    model.compile(
      optimizer=optimizer,
      loss='categorical_crossentropy',
      metrics=['accuracy'],
    )
    return model

In [4]:
model = create_model()

# Train the model.
model.fit(train_images, to_categorical(train_labels), epochs=5, batch_size=32)

# Evaluate the model.
loss,metrics = model.evaluate(test_images, to_categorical(test_labels))
print('*****************************')
print(f'Loss - {np.round(loss,2)}, Metrics - {np.round(metrics,2)}')

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2923 - accuracy: 0.9152
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1341 - accuracy: 0.9599
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0981 - accuracy: 0.9700
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0776 - accuracy: 0.9764
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.0952 - accuracy: 0.9722
*****************************
Loss - 0.1, Metrics - 0.97


In [5]:
# Predict on the first 10 test images.
predictions = model.predict(test_images[:10])

# Print our model's predictions.
print(f'Predictions: {np.argmax(predictions, axis=1)}')

# Check our predictions against the ground truths.
print(f'True labels: {test_labels[:10]}')

1/1 [==============================] - 0s 83ms/step
Predictions: [7 2 1 0 4 1 4 9 5 9]
True labels: [7 2 1 0 4 1 4 9 5 9]


### Picking params via GridSearchCV

In [6]:
batch_size = [100, 300]
epochs = [5, 10, 20]
optimizer = ['SGD', 'RMSprop', 'Adam','AdamW','Adadelta','Adagrad','Adamax','Adafactor','Nadam','Ftrl']

In [7]:
model = KerasClassifier(create_model)
params= dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer)
grid = GridSearchCV(model, params, refit=False, cv=3, scoring='accuracy')

grid_result = grid.fit(train_images, to_categorical(train_labels))
print(grid_result.best_score_, grid_result.best_params_)

Epoch 1/5
400/400 [==============================] - 1s 2ms/step - loss: 0.4705 - accuracy: 0.8701
Epoch 2/5
400/400 [==============================] - 1s 2ms/step - loss: 0.1993 - accuracy: 0.9424
Epoch 3/5
400/400 [==============================] - 1s 2ms/step - loss: 0.1559 - accuracy: 0.9547
Epoch 4/5
400/400 [==============================] - 1s 2ms/step - loss: 0.1255 - accuracy: 0.9634
Epoch 5/5
200/200 [==============================] - 0s 1ms/step
Epoch 1/5
400/400 [==============================] - 1s 2ms/step - loss: 0.4788 - accuracy: 0.8594
Epoch 2/5
400/400 [==============================] - 1s 2ms/step - loss: 0.1957 - accuracy: 0.9434
Epoch 3/5
400/400 [==============================] - 1s 2ms/step - loss: 0.1479 - accuracy: 0.9578
Epoch 4/5
400/400 [==============================] - 1s 2ms/step - loss: 0.1190 - accuracy: 0.9657
Epoch 5/5
200/200 [==============================] - 0s 1ms/step
Epoch 1/5
400/400 [==============================] - 1s 2ms/step - loss: 0.458

In [8]:
print(grid_result.best_score_, grid_result.best_params_)

0.9666166666666666 {'batch_size': 100, 'epochs': 10, 'optimizer': 'Adafactor'}


In [9]:
df = pd.DataFrame(grid_result.cv_results_)

In [20]:

df.sort_values(['rank_test_score'],ascending=False)[['rank_test_score','param_batch_size','param_epochs','param_optimizer','mean_test_score']].head(5)

,rank_test_score,param_batch_size,param_epochs,param_optimizer,mean_test_score
21,40,300,5,RMSprop,0.949850
25,39,300,5,Adagrad,0.949983
29,38,300,5,Ftrl,0.950267
28,37,300,5,Nadam,0.950533
24,36,300,5,Adadelta,0.950567


In [18]:
df.sort_values(['mean_test_score'],ascending=False)[['param_batch_size','param_epochs','param_optimizer','mean_test_score','rank_test_score']].head(20)

,param_batch_size,param_epochs,param_optimizer,mean_test_score,rank_test_score
17,100,10,Adafactor,0.966617,1
13,100,10,AdamW,0.966317,2
14,100,10,Adadelta,0.966250,3
19,100,10,Ftrl,0.965867,4
10,100,10,SGD,0.965783,5
18,100,10,Nadam,0.965700,6
16,100,10,Adamax,0.965133,7
11,100,10,RMSprop,0.965083,8
15,100,10,Adagrad,0.965033,9
12,100,10,Adam,0.964733,10


Оказалось что на текущих параметрах обучения увеличение размера батча негативно влияет на точность нейросети. Все оптимизаторы показали приближенные друг к другу результаты. Максимальная разница наодинаковых параметрах ~0.0019. На 10эпохахлучше всего себя показал Adafactor а на 5 Adam который отстает от них на ~0.0025. Худжие результаты ыли достигнуты намаксимальном размере атча и минимальном количестве эпох. Разница между 1 и оследним местом ~0.017 что уже существенно